In [2]:
import datetime
import os
import requests
import pandas as pd
import importlib
import zipfile
from tqdm import tqdm_notebook as tqdm
import glob
from bs4 import BeautifulSoup
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser

In [4]:
code_and_return = pd.read_csv('code_and_return.csv')
code_and_return

,code,return
0,13010,0.094946
1,13050,0.087043
2,13060,0.087250
3,13080,0.086220
4,13090,-0.009877
...,...,...
3992,99930,-0.158866
3993,99940,0.080080
3994,99950,-0.007264
3995,99960,-0.067485


In [5]:
def specify_time_period():
    start_date = datetime.date(2022, 3, 31)
    end_date = datetime.date(2022, 6, 30)
    #決算後3ヶ月以内に有報を提出しなければならない規則から、end_dateを2022/06/30に指定

    period = end_date - start_date
    period = int(period.days)
    day_list = []
    for d in range(period):
        day = start_date + datetime.timedelta(days=d)
        day_list.append(day)
        
    day_list.append(end_date)

    return day_list


In [6]:
def get_report_list(day_list, codes):
    #結果を格納するための空のリストを用意
    report_list =[]
    #日付リストの期間に提出された書類のメタデータを取得してjson形式に変換
    for day in day_list:
        url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents.json"
        params = {"date": day, "type": 2}
        print(day)
        res = requests.get(url, params=params)
        print(res)
        if res.status_code == 200:
            try:
                json_data = res.json()
            except ValueError:
                print("Invalid JSON format")
        else:
            print("Request failed with status code:", res.status_code)
        for num in range(len(json_data["results"])):
            ordinance_code = json_data["results"][num]["ordinanceCode"]
            form_code = json_data["results"][num]["formCode"]
            sec_code = json_data["results"][num]["secCode"]
            
        #ordinance_code=010かつform_code=030000が有価証券報告書
            if ordinance_code == "010" and form_code =="030000" and sec_code in codes:
                company_name=json_data["results"][num]["filerName"]
                edi={ '会社名':company_name,
                            '書類名':json_data["results"][num]["docDescription"],
                            'docID':json_data["results"][num]["docID"],
                            '証券コード':json_data["results"][num]["secCode"],
                            '開始時期':json_data["results"][num]["periodStart"],
                            '終了時期':json_data["results"][num]["periodEnd"],
                            '日付': day,
                            'XBRL':json_data["results"][num]["xbrlFlag"]}
                report_list.append(edi)

    return report_list

In [7]:
def download_zip(docid_list, save_dir):
    filename_list = []


    for docid in tqdm(docid_list):
        # 書類取得APIのエンドポイント
        url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents/" + docid
        #print(url)
        
        # 書類取得APIのリクエストパラメータ
        params = {
            "type" : 1}
            
        # 出力ファイル名
        filename = docid + ".zip"
        filename_list.append(filename)

        # 出力ファイルパス
        save_path = os.path.join(save_dir, filename)

        # 書類取得APIの呼び出し
        res = requests.get(url, params=params, verify=False)
        # ファイルへ出力
        # print(res.status_code)
        if res.status_code == 200:
            with open(save_path, 'wb') as f:
                for chunk in res.iter_content(chunk_size=1024):
                    f.write(chunk)

    return filename_list

In [11]:
def extract_zip(docid_list, save_dir, filename_list):
    extract_dir = 'sequrity_reports_for_training/'

    for i, docid in enumerate(docid_list):
        # 出力ファイルパス
        file_path = os.path.join(save_dir, filename_list[i])

        print(file_path)
        
        if zipfile.is_zipfile(file_path) == True:
            with zipfile.ZipFile(file_path) as zip_f:
                # 解凍先ディレクトリを指定して解凍
                extract_path = os.path.join(extract_dir, docid)
                os.makedirs(extract_path, exist_ok=True)
                zip_f.extractall(extract_path)
        else:  # エラー回避
            print(file_path + " is not a Zip File")
            docid_list.remove(docid)

In [9]:
int_codes = code_and_return['code'].tolist()
codes = [str(num) for num in int_codes]


day_list = specify_time_period()
report_list = get_report_list(day_list, codes)

2022-03-31
<Response [200]>
2022-04-01
<Response [200]>
2022-04-02
<Response [200]>
2022-04-03
<Response [200]>
2022-04-04
<Response [200]>
2022-04-05
<Response [200]>
2022-04-06
<Response [200]>
2022-04-07
<Response [200]>
2022-04-08
<Response [200]>
2022-04-09
<Response [200]>
2022-04-10
<Response [200]>
2022-04-11
<Response [200]>
2022-04-12
<Response [200]>
2022-04-13
<Response [200]>
2022-04-14
<Response [200]>
2022-04-15
<Response [200]>
2022-04-16
<Response [200]>
2022-04-17
<Response [200]>
2022-04-18
<Response [200]>
2022-04-19
<Response [200]>
2022-04-20
<Response [200]>
2022-04-21
<Response [200]>
2022-04-22
<Response [200]>
2022-04-23
<Response [200]>
2022-04-24
<Response [200]>
2022-04-25
<Response [200]>
2022-04-26
<Response [200]>
2022-04-27
<Response [200]>
2022-04-28
<Response [200]>
2022-04-29
<Response [200]>
2022-04-30
<Response [200]>
2022-05-01
<Response [200]>
2022-05-02
<Response [200]>
2022-05-03
<Response [200]>
2022-05-04
<Response [200]>
2022-05-05
<Response

In [10]:
print(len(report_list))

1054


In [12]:
df = pd.DataFrame(report_list)
print("----df----")
print(df['証券コード'])


docid_list = df['docID'].tolist()

print("----docid----")
print(docid_list)
save_dir = 'zip_for_training/'

filename_list = download_zip(docid_list, save_dir)

extract_zip(docid_list, save_dir, filename_list)

----df----
0       77160
1       76090
2       59650
3       59460
4       69770
        ...  
1049    25880
1050    72640
1051    36040
1052    72960
1053    37990
Name: 証券コード, Length: 1054, dtype: object
----docid----
['S100NRFI', 'S100NR8K', 'S100NST4', 'S100NSKK', 'S100NSFA', 'S100NSS0', 'S100NSJH', 'S100NRNF', 'S100NSSQ', 'S100NSVK', 'S100NPM4', 'S100NSCG', 'S100NR98', 'S100NRML', 'S100NSW0', 'S100NQY7', 'S100NRXY', 'S100NQRA', 'S100NST7', 'S100NSWK', 'S100NSPY', 'S100NSR5', 'S100NSVW', 'S100NMHP', 'S100NQ20', 'S100NRK1', 'S100NSP9', 'S100NSI7', 'S100NQC5', 'S100NSUF', 'S100NRBE', 'S100NSY7', 'S100NSKW', 'S100NSY4', 'S100NSG3', 'S100NSTC', 'S100NSZM', 'S100NRXZ', 'S100NSXJ', 'S100NSTP', 'S100NRPZ', 'S100NS8P', 'S100NSQZ', 'S100NSV5', 'S100NSRY', 'S100NSDW', 'S100NT0X', 'S100NT1F', 'S100NSZ0', 'S100NT1A', 'S100NT0S', 'S100NT2U', 'S100NT5B', 'S100NT5U', 'S100NT5C', 'S100NSYQ', 'S100NSYB', 'S100NSZD', 'S100NSYN', 'S100NS0A', 'S100NT3I', 'S100NT29', 'S100NSQC', 'S100NSSV', 'S100NS97',

<ipython-input-7-29ab0780bfcd>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for docid in tqdm(docid_list):


  0%|          | 0/1054 [00:00<?, ?it/s]

/Users/hiroki/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'disclosure.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/hiroki/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'disclosure.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/hiroki/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'disclosure.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings


zip_for_training/S100NRFI.zip
zip_for_training/S100NR8K.zip
zip_for_training/S100NST4.zip
zip_for_training/S100NSKK.zip
zip_for_training/S100NSFA.zip
zip_for_training/S100NSS0.zip
zip_for_training/S100NSJH.zip
zip_for_training/S100NRNF.zip
zip_for_training/S100NSSQ.zip
zip_for_training/S100NSVK.zip
zip_for_training/S100NPM4.zip
zip_for_training/S100NSCG.zip
zip_for_training/S100NR98.zip
zip_for_training/S100NRML.zip
zip_for_training/S100NSW0.zip
zip_for_training/S100NQY7.zip
zip_for_training/S100NRXY.zip
zip_for_training/S100NQRA.zip
zip_for_training/S100NST7.zip
zip_for_training/S100NSWK.zip
zip_for_training/S100NSPY.zip
zip_for_training/S100NSR5.zip
zip_for_training/S100NSVW.zip
zip_for_training/S100NMHP.zip
zip_for_training/S100NQ20.zip
zip_for_training/S100NRK1.zip
zip_for_training/S100NSP9.zip
zip_for_training/S100NSI7.zip
zip_for_training/S100NQC5.zip
zip_for_training/S100NSUF.zip
zip_for_training/S100NRBE.zip
zip_for_training/S100NSY7.zip
zip_for_training/S100NSKW.zip
zip_for_tr

In [14]:
parser = EdinetXbrlParser()


content = pd.DataFrame(columns=['docID','経営方針'])


for _docID in tqdm(docid_list):
    #xbrlファイルのパスを指定
    xbrl_path = 'sequrity_reports_for_training/' + _docID + '/XBRL/PublicDoc/*.xbrl'
    # print(i)
    # print(xbrl_path)
    # print(glob.glob(xbrl_path))
    xbrl_path = glob.glob(xbrl_path)[0]
    # print(xbrl_path)
    edinet_xbrl_object = parser.parse_file(xbrl_path)

    #タクソノミ要素となるkeyとcontext_refを指定
    #経営方針、経営環境および対処すべき課題等
    key_houshin= 'jpcrp_cor:BusinessPolicyBusinessEnvironmentIssuesToAddressEtcTextBlock'

    #context_refを定義
    context_ref_1 = 'FilingDateInstant'
        
    #例外処理
    try:
        data_houshin = edinet_xbrl_object.get_data_by_context_ref(key_houshin, context_ref_1).get_value()
        soup_houshin = BeautifulSoup(data_houshin,'html.parser').get_text(strip=True)
        
        tmp_se = pd.DataFrame([[_docID], [soup_houshin]], index=content.columns).T
        content = content.append(tmp_se)

    except AttributeError as e:
        print(e)

<ipython-input-14-41d25da8a73c>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _docID in tqdm(docid_list):


  0%|          | 0/1054 [00:00<?, ?it/s]

/Users/hiroki/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
<ipython-input-14-41d25da8a73c>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  content = content.append(tmp_se)
/Users/hiroki/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning.

In [15]:
content

,docID,経営方針
0,S100NRFI,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...
0,S100NR8K,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...
0,S100NST4,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...
0,S100NSKK,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...
0,S100NSFA,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末におい...
...,...,...
0,S100OC2Y,１【経営方針、経営環境及び対処すべき課題等】当社グループの事業を取り巻く環境は、当面は新型コ...
0,S100OEDJ,１ 【経営方針、経営環境及び対処すべき課題等】(1) 経営方針①経営の基本方針当社は、ものづ...
0,S100OEK8,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...
0,S100OE8J,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、有価証券報告書提出日現...


In [16]:
content = content.reset_index(drop=True)
content

,docID,経営方針
0,S100NRFI,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...
1,S100NR8K,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...
2,S100NST4,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...
3,S100NSKK,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...
4,S100NSFA,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末におい...
...,...,...
1049,S100OC2Y,１【経営方針、経営環境及び対処すべき課題等】当社グループの事業を取り巻く環境は、当面は新型コ...
1050,S100OEDJ,１ 【経営方針、経営環境及び対処すべき課題等】(1) 経営方針①経営の基本方針当社は、ものづ...
1051,S100OEK8,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...
1052,S100OE8J,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、有価証券報告書提出日現...


In [17]:
df['証券コード']

0       77160
1       76090
2       59650
3       59460
4       69770
        ...  
1049    25880
1050    72640
1051    36040
1052    72960
1053    37990
Name: 証券コード, Length: 1054, dtype: object

In [18]:
content['code'] = df['証券コード'].astype('int')
content = content.drop('docID', axis=1)

print("----content2----")
print(content)

code_and_return_and_content = pd.merge(content, code_and_return, on='code', how='inner')

----content2----
                                                   経営方針   code
0     １【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...  77160
1     １【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...  76090
2     １ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...  59650
3     １【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...  59460
4     １【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末におい...  69770
...                                                 ...    ...
1049  １【経営方針、経営環境及び対処すべき課題等】当社グループの事業を取り巻く環境は、当面は新型コ...  25880
1050  １ 【経営方針、経営環境及び対処すべき課題等】(1) 経営方針①経営の基本方針当社は、ものづ...  72640
1051  １【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...  36040
1052  １【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、有価証券報告書提出日現...  72960
1053  １ 【経営方針、経営環境及び対処すべき課題等】当社および当社の子会社(以下「当社グループ」と...  37990

[1054 rows x 2 columns]


In [19]:
code_and_return_and_content

,経営方針,code,return
0,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...,77160,0.209313
1,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...,76090,0.377778
2,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...,59650,-0.063342
3,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...,59460,0.282051
4,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末におい...,69770,0.145766
...,...,...,...
1049,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、有価証券報告書提出日現...,72960,0.342813
1050,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、有価証券報告書提出日現...,72960,0.342813
1051,１ 【経営方針、経営環境及び対処すべき課題等】当社および当社の子会社(以下「当社グループ」と...,37990,-0.030928
1052,１ 【経営方針、経営環境及び対処すべき課題等】当社および当社の子会社(以下「当社グループ」と...,37990,-0.030928


In [20]:
code_and_return_and_content.to_csv('../datasets/datasets_for_training.csv', index=False)